<a href="https://colab.research.google.com/github/SLVmain/NLP/blob/Les_13/HW_13_BERT_para.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Урок 13. Модель BERT и GPT
###Задание

Взять датасет
https://huggingface.co/datasets/merionum/ru_paraphraser
решить задачу парафраза

#Задача парафраз

In [1]:
!pip install datasets transformers SentencePiece folium==0.2.1 --quiet

     |████████████████████████████████| 362 kB 5.1 MB/s 
     |████████████████████████████████| 4.4 MB 59.0 MB/s 
     |████████████████████████████████| 1.2 MB 65.5 MB/s 
     |████████████████████████████████| 69 kB 8.8 MB/s 
     |████████████████████████████████| 140 kB 69.6 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 212 kB 76.1 MB/s 
     |████████████████████████████████| 1.1 MB 78.6 MB/s 
     |████████████████████████████████| 596 kB 60.4 MB/s 
     |████████████████████████████████| 127 kB 55.8 MB/s 
     |████████████████████████████████| 6.6 MB 78.4 MB/s 
     |████████████████████████████████| 271 kB 67.6 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 144 kB 68.5 MB/s 


In [2]:
from datasets import load_dataset

corpus = load_dataset("merionum/ru_paraphraser")

Using custom data configuration merionum--ru_paraphraser-1a7592429d7be082


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-1a7592429d7be082/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [4]:
import torch
torch.cuda.empty_cache()
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")

model = AutoModelForSeq2SeqLM.from_pretrained("cointegrated/rut5-base-paraphraser")

model.cuda();

Downloading:   0%|          | 0.00/315 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/808k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/convert_slow_tokenizer.py:435: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"


Downloading:   0%|          | 0.00/724 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/932M [00:00<?, ?B/s]

In [5]:
def paraphrase(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [6]:
for i in range(10,21):
    print('Исходный текст: ', corpus["train"][i]['text_1'])
    print('Парафраз: ', paraphrase(corpus["train"][i]['text_1']))
    print('------------------------------------------------------------')

Исходный текст:  Москвичи смогут забронировать в Интернете место на кладбище.
Парафраз:  Москвичи могут забронировать место в интернете на кладбищах.
------------------------------------------------------------
Исходный текст:  Въезд в центр Москвы автомобилям с двигателями Евро-2 не запрещали.
Парафраз:  В центре Москвы не запрещалось въезд автомобилей с двигателем Евро-2.
------------------------------------------------------------
Исходный текст:  В Подмосковье произошел пожар на железной дороге.
Парафраз:  Пожар на железнодорожной дороге произошел в Подмосковье.
------------------------------------------------------------
Исходный текст:  Северокорейский лидер впервые за 19 лет поздравил граждан с Новым годом.
Парафраз:  Впервые в 19 лет лидер Северокорейской Республики поздравил жителей с Новым Годом.
------------------------------------------------------------
Исходный текст:  Мужчина из Подмосковья случайно убил жену в Новый год.
Парафраз:  Мужчина в Подмосковье случайно убила ж

In [ ]:
#в какой-то момент оперативной памяти не хватило, но частично дообучить получилось
optimizer = torch.optim.Adam(params=[p for p in model.parameters() if p.requires_grad], lr=1e-5)

for i in range(10):
    x = tokenizer(corpus['train']['text_1'][:100], return_tensors='pt', padding=True).to(model.device)
    y = tokenizer(corpus['train']['text_2'][:100], return_tensors='pt', padding=True).to(model.device)

    y.input_ids[y.input_ids==0] = -100
    loss = model(
        input_ids=x.input_ids,
        attention_mask=x.attention_mask,
        labels=y.input_ids,
        decoder_attention_mask=y.attention_mask,
        return_dict=True
    ).loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
model.save_pretrained('paraphraser_tuned')
tokenizer.save_pretrained('paraphraser_tuned')

In [16]:
model1=model.from_pretrained('paraphraser_tuned')
model1.cuda()
def paraphrase1(text, beams=5, grams=4, do_sample=False):
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    out = model1.generate(**x, encoder_no_repeat_ngram_size=grams, num_beams=beams, max_length=max_size, do_sample=do_sample)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [18]:
for i in range(10,21):
    print('Исходный текст: ', corpus["train"][i]['text_1'])
    print('Парафраз без tuning: ', paraphrase(corpus["train"][i]['text_1']))
    print('Парафраз с tuning: ', paraphrase1(corpus["train"][i]['text_1']))
    print('------------------------------------------------------------')

Исходный текст:  Москвичи смогут забронировать в Интернете место на кладбище.
Парафраз без tuning:  Москвичи могут забронировать место в интернете.
Парафраз с tuning:  Москвичи могут забронировать место в интернете.
------------------------------------------------------------
Исходный текст:  Въезд в центр Москвы автомобилям с двигателями Евро-2 не запрещали.
Парафраз без tuning:  В центре Москвы не запретили въезд автомобилей с двигателем Евро-2.
Парафраз с tuning:  В центре Москвы не запретили въезд автомобилей с двигателем Евро-2.
------------------------------------------------------------
Исходный текст:  В Подмосковье произошел пожар на железной дороге.
Парафраз без tuning:  Пожар на железнодорожной дороге произошел в Подмосковье
Парафраз с tuning:  Пожар на железнодорожной дороге произошел в Подмосковье
------------------------------------------------------------
Исходный текст:  Северокорейский лидер впервые за 19 лет поздравил граждан с Новым годом.
Парафраз без tuning:  Вперв

неплохо модель справляется с задачей, дообучение ничего не улучшило в данном случае, возможно, нужно больше дообучать, но были проблемы с оперативной памятью